## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
num_classes

2

## Creating a new model

In [ ]:
model = Sequential()
model.add(VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3211392   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 23,235,905
Trainable params: 3,211,521
Non-trainable params: 20,024,384
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-VGG19-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-VGG19-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1279s 4s/step - loss: 3.9374 - accuracy: 0.7032 - mean_absolute_error: 9.7573 - mean_squared_error: 367.1134 - hinge: 3.9211 - binary_crossentropy: 4.1960 - val_loss: 0.2724 - val_accuracy: 0.9200 - val_mean_absolute_error: 4.1206 - val_mean_squared_error: 25.4427 - val_hinge: 0.2584 - val_binary_crossentropy: 1.0110
Epoch 2/20
300/300 [==============================] - 28s 92ms/step - loss: 0.3271 - accuracy: 0.8945 - mean_absolute_error: 4.7859 - mean_squared_error: 35.6519 - hinge: 0.3138 - binary_crossentropy: 1.3194 - val_loss: 0.2498 - val_accuracy: 0.8900 - val_mean_absolute_error: 3.2746 - val_mean_squared_error: 15.5273 - val_hinge: 0.2383 - val_binary_crossentropy: 1.3243
Epoch 3/20
300/300 [==============================] - 28s 93ms/step - loss: 0.1452 - accuracy: 0.9490 - mean_absolute_error: 3.7575 - mean_squared_error: 21.1448 - hinge: 0.1342 - binary_crossentropy: 0.5652 - val_loss: 0.3180 - val_accuracy: 0.8464 - val